# Pandas

Pandas performs in memory computation, and often stores copies during calculations. 
Rule of thumb: pandas works well for datasets under 10GB



In [1]:
import pandas as pd

# row wise operations
worst to best:

In [2]:
df = pd.DataFrame({"a": list(range(1000)),
                    "b": list(range(1000)),
                    "c": list(range(1000))
                    })


In [3]:
%%timeit

# worst: have to do lookups and 
# row is a newly constructed series
for i in range(len(df)):
    row = df.iloc[i] * 2
    df.iloc[i] = row

203 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit

# slightly better because no lookups but
# row still is a newly constructed series
for i, row in df.iterrows():
    df.iloc[i] = row * 2

165 ms ± 488 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit

# slightly better because no need for
# intermediate objects, although it does still 
# construct a new series under the hood
df.apply(lambda row: row * 2, axis=1)

88.5 ms ± 645 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### The raw=True flag

Use raw=True for numerical operations over DataFrames where pandas Series features are not needed. For operations that rely on pandas-specific features or the index, leave raw at its default (False).

In [6]:
%%timeit

# raw=true will stop the creation of new series
# under the hood and pass the original values
df.apply(lambda row: row * 2, axis=1, raw=True)

2.21 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Avoid concat like the plague

In [7]:
%%timeit
result = pd.DataFrame({"a": [], "b": []})
for i in range(100):
    result = pd.concat([result, pd.DataFrame({"a": [i], "b": [i]})])

48.5 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
result = []
for i in range(100):
    result.append({"a": i, "b": i})
result = pd.DataFrame(result)

303 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Catagory dtype
for large dataframe with catagories with low cardinality (e.g. is_deleted: yes/no, control_type: vfd, dol etc.)
change the dtype to catagory. It will speed up operations such as value_counts and group_by

In [9]:
import numpy as np
df = pd.DataFrame({"control_type": np.random.choice(["VFD", "DOL", "UNKNOWN"], 100_000)})

In [10]:
df.dtypes

control_type    object
dtype: object

In [11]:
%%timeit
df["control_type"].value_counts()

5.85 ms ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
df.groupby("control_type").count()

6.2 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
df["control_type"] = df["control_type"].astype("category")

In [14]:
%%timeit
df["control_type"].value_counts()

593 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%%timeit
df.groupby("control_type").count()

629 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
